# Notebook 1: Extract Z-scored LFP

### Overview
This Jupyter notebook focuses on extracting local field potential (LFP) traces from Spikegadgets `.rec` files, specifically for neuroscience research related to social competition trials. The notebook includes procedures for preprocessing and synchronizing raw electrophysiology data with corresponding video data and computes various metrics, including Z-scored LFPs.

### Inputs & Data Sources
- **Electrophysiology and LFP Parameters**: Constants like `EPHYS_SAMPLING_RATE`, `LFP_SAMPLING_RATE`, `TRIAL_DURATION`, etc., define basic parameters for LFP data processing.
- **Recording Information**: Stream IDs (`ECU_STREAM_ID`, `TRODES_STREAM_ID`), recording extension (`RECORDING_EXTENSION`), and paths to recording directories (`ALL_SESSION_DIR`).
- **DataFrames for Mapping and Timestamps**: `CHANNEL_MAPPING_DF` for channel mapping, and `TONE_TIMESTAMP_DF` for tone timestamps, loaded from external sources.
- **Constants for DataFrame Columns**: Names for various columns in the DataFrame, defined in an all-caps snake case format, such as `EPHYS_INDEX_COL`, `LFP_INDEX_COL`, etc.

### Output & Utility
- **Processed Data**: The notebook outputs processed data, particularly the Z-scored LFP traces, which are critical for further analysis in neuroscience research.
- **Data Files**: Outputs are saved in various formats (`CSV`, `Pickle`) in a specified output directory (`OUTPUT_DIR`).
- **Visualization**: While not explicitly mentioned, the notebook has the potential for data visualization (plots) based on processed LFP data.

### Processing Workflow
1. **LFP Extraction and Preprocessing**: 
    - Iterates through recording sessions to process `.rec` files.
    - Applies a series of preprocessing steps like bandpass filtering, notch filtering, resampling, and Z-scoring on the LFP data.
    - Exception handling for cases where the recording doesn't contain specified stream IDs.

2. **DataFrame Manipulation and Merging**:
    - Filtering `TONE_TIMESTAMP_DF` for trials with obtained LFP.
    - Addition of trial numbers and merging with `CHANNEL_MAPPING_DF`.
    - Dropping unnecessary columns and restructuring for analysis.

3. **LFP Trace Extraction for Each Trial and Brain Region**: 
    - Linking LFP calculations with trials.
    - Creating new rows for each brain region, extracting baseline, trial, and combined LFP traces.
    - Results in a comprehensive DataFrame that combines trial information with corresponding LFP traces.

4. **Data Storage**:
    - Saving processed DataFrame in both `CSV` and `Pickle` formats for easy access and future use.

### Usage Notes
- The notebook is project-specific and tailored for a particular dataset structure, requiring modifications for different data formats.
- Users should ensure file paths and directory names match their project's structure and adjust constants and parameters as needed for their specific analysis requirements.
- The notebook forms a part of a larger research framework, thus necessitating compatibility checks with other components of the project.

### Dependencies
- Python Libraries: `sys`, `os`, `glob`, `numpy`, `pandas`, `spikeinterface`
- External Data: Channel mapping and tone timestamp files, along with Spikegadgets `.rec` files.

### Customization and Scalability
- The notebook's modular design allows for easy adaptation to different datasets or extensions to include additional processing steps.
- Functions and processing steps are clearly demarcated, facilitating straightforward updates or enhancements.

### Conclusion
This notebook is a vital tool in the preprocessing and analysis of LFP data from Spikegadgets recordings, integral to neuroscience research focused on social competition trials. It offers a structured approach to handle, process, and store electrophysiological data, ensuring reproducibility and efficiency in research workflows.

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [9]:
EPHYS_INDEX_COL = "time_stamp_index"
LFP_INDEX_COL = "lfp_index"
EPHYS_TIMESTAMP_COL = "time"
RECORDING_FILE_COL = "recording_file"
RECORDING_DIR_COL = "recording_dir"
BASELINE_LFP_INDEX_RANGE_COL = "baseline_lfp_index_range"
TRIAL_LFP_INDEX_RANGE_COL = "trial_lfp_index_range"
BASELINE_EPHYS_INDEX_RANGE_COL = "baseline_ephys_index_range"
TRIAL_EPHYS_INDEX_RANGE_COL = "trial_ephys_index_range"
BASELINE_VIDEOFRAME_RANGE_COL = "baseline_videoframe_range"
TRIAL_VIDEOFRAME_RANGE_COL = "trial_videoframe_range"
CURRENT_SUBJECT_COL = "current_subject"
ALL_CH_LFP_COL = "all_ch_lfp"
SUBJECT_COL = "Subject"
TRIAL_NUMBER_COL = "trial_number"
SPIKE_INTERFACE_COL = "spike_interface"
EIB_COL = "eib"

In [10]:
# NOTE: Change based on individual project data location

# Spreadsheet of channel mapping
CHANNEL_MAPPING_DF = pd.read_excel(os.path.join(git_root, "data/channel_mapping.xlsx"))
# Spreadsheet of tone time
SPIKEGADGETS_EXTRACTED_DF = pd.read_pickle("./proc/rce_pilot_2_trodes_metadata.pkl")

In [11]:
CHANNEL_MAPPING_DF.head()

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0


In [12]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_16/*.rec")

In [13]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/standard/2023_06_16/20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [15]:
TONE_TIMESTAMPS_CSV = "rce_tone_timestamps.csv"
TONE_TIMESTAMPS_PKL = "rce_tone_timestamps.pkl"
FULL_LFP_TRACES_PKL = "full_baseline_and_trial_lfp_traces.pkl"

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [16]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Extracting the LFP

In [17]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero
    




An exception occurred: stream_id trodes is not in ['ECU']
20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged
20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged


## Adding the channel mapping

In [18]:
CHANNEL_MAPPING_DF = CHANNEL_MAPPING_DF.drop(columns=[col for col in CHANNEL_MAPPING_DF.columns if "eib" in col], errors="ignore")

In [19]:
for col in CHANNEL_MAPPING_DF.columns:
    if "spike_interface" in col:
        CHANNEL_MAPPING_DF[col] = CHANNEL_MAPPING_DF[col].fillna(0)
        CHANNEL_MAPPING_DF[col] = CHANNEL_MAPPING_DF[col].astype(int).astype(str)

In [20]:
CHANNEL_MAPPING_DF

,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,21,15,14,13,16
1,1,6.2,0,0,0,0,0
2,1,6.3,0,0,0,0,0
3,1,6.4,0,0,0,0,0
4,2,1.1,5,31,30,29,28
5,2,1.2,10,31,30,29,28
6,2,1.3,9,31,30,29,28
7,2,1.4,15,31,30,29,28


- Adding all the brain region to ch information

In [21]:
CHANNEL_MAPPING_DF[SUBJECT_COL] = CHANNEL_MAPPING_DF[SUBJECT_COL].astype(str)

- Merging the recording and the channel dataframes

In [22]:
SPIKEGADGETS_EXTRACTED_DF.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,time,timestamps,Timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",systime,"[1686928758076131800, 1686928758076134800, 168...",[1.4],1.4
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,coordinates,Pad locations in microns,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,ml,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ap,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[1.4],1.4
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Dout3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Dout3,4,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4


In [23]:
SPIKEGADGETS_EXTRACTED_DF = pd.merge(SPIKEGADGETS_EXTRACTED_DF, CHANNEL_MAPPING_DF, left_on=CURRENT_SUBJECT_COL, right_on=SUBJECT_COL, how="left")



In [24]:
SPIKEGADGETS_EXTRACTED_DF.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,last_item_data,all_subjects,current_subject,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,time,timestamps,Timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[1686928758076131800, 1686928758076134800, 168...",[1.4],1.4,2.0,1.4,15,31,30,29,28
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4,2.0,1.4,15,31,30,29,28
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,coordinates,Pad locations in microns,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[1.4],1.4,2.0,1.4,15,31,30,29,28
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4,2.0,1.4,15,31,30,29,28
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Dout3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,[0],[1.4],1.4,2.0,1.4,15,31,30,29,28


In [25]:
SPIKEGADGETS_EXTRACTED_DF[["metadata_dir", "metadata_file"]]

,metadata_dir,metadata_file
0,time,timestamps
1,raw,timestamps
2,raw,coordinates
3,analog,timestamps
4,DIO,dio_ECU_Dout3
5,DIO,dio_ECU_Din3
6,DIO,dio_ECU_Din2
7,DIO,dio_ECU_Din1
8,DIO,dio_ECU_Dout2
9,DIO,dio_ECU_Dout4


# Getting the channel specific LFP traces

- Linking up all LFP calculations with all the trials

In [26]:
SPIKEGADGETS_EXTRACTED_DF[SPIKEGADGETS_EXTRACTED_DF["metadata_file"] == "timestamps"]

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,last_item_data,all_subjects,current_subject,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,time,timestamps,Timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[1686928758076131800, 1686928758076134800, 168...",[1.4],1.4,2.0,1.4,15,31,30,29,28
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4,2.0,1.4,15,31,30,29,28
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4,2.0,1.4,15,31,30,29,28
12,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,time,timestamps,Timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[1686928758076131800, 1686928758076134800, 168...",[1.2],1.2,2.0,1.2,10,31,30,29,28
22,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.2],1.2,2.0,1.2,10,31,30,29,28
23,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.2],1.2,2.0,1.2,10,31,30,29,28


In [34]:
condition = (SPIKEGADGETS_EXTRACTED_DF['metadata_dir'] == 'raw') & (SPIKEGADGETS_EXTRACTED_DF['metadata_file'] == 'timestamps')

In [35]:
condition

0     False
1      True
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22     True
23    False
24    False
dtype: bool

In [36]:
SPIKEGADGETS_EXTRACTED_DF[ALL_CH_LFP_COL] = SPIKEGADGETS_EXTRACTED_DF["recording"].where(condition).map(recording_name_to_all_ch_lfp)

In [38]:
SPIKEGADGETS_EXTRACTED_DF[ALL_CH_LFP_COL]

0                                                   NaN
1     ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
2                                                   NaN
3                                                   NaN
4                                                   NaN
5                                                   NaN
6                                                   NaN
7                                                   NaN
8                                                   NaN
9                                                   NaN
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [33]:
raise ValueError("stop")

ValueError: stop

In [ ]:
SPIKEGADGETS_EXTRACTED_DF[ALL_CH_LFP_COL] = SPIKEGADGETS_EXTRACTED_DF[RECORDING_FILE_COL].map(recording_name_to_all_ch_lfp)



In [ ]:
SPIKEGADGETS_EXTRACTED_DF

- Extracting the traces for each brain region

In [ ]:
for col in columns_to_convert:
    print(col)
    brain_region = col.strip(SPIKE_INTERFACE_COL).strip("_")
    trace_column = "{}_lfp_trace".format(brain_region)
    SPIKEGADGETS_EXTRACTED_DF[trace_column] = SPIKEGADGETS_EXTRACTED_DF.apply(lambda row: row[ALL_CH_LFP_COL].get_traces(channel_ids=[row[col]]).T[0], axis=1)
                                                                                                                                                       
                                                                                                                                                    

In [ ]:
SPIKEGADGETS_EXTRACTED_DF = SPIKEGADGETS_EXTRACTED_DF.drop(columns=[ALL_CH_LFP_COL], errors="ignore")

In [ ]:
SPIKEGADGETS_EXTRACTED_DF = SPIKEGADGETS_EXTRACTED_DF.drop(columns=[col for col in SPIKEGADGETS_EXTRACTED_DF if SPIKE_INTERFACE_COL in col], errors="ignore")

In [ ]:
SPIKEGADGETS_EXTRACTED_DF.head()

In [ ]:
SPIKEGADGETS_EXTRACTED_DF.tail()

In [ ]:
SPIKEGADGETS_EXTRACTED_DF.columns

In [ ]:
SPIKEGADGETS_EXTRACTED_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))